## PROJECT 1

#### Dave Bletsch

#### Metacritic webscrape

### Import packages for webscraping

In [1]:
# to help bypass 403 and 429 error codes
from urllib.request import Request, urlopen

# for scraping
from bs4 import BeautifulSoup as BS

# formatting date 
from datetime import datetime

# for pausing in between changing urls
import time

In [12]:
# list of years to insert into search URL
years = list(range(2019, 2009, -1))
years

[2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010]

### Scrape Metacritic site for top 100 movies each of the past 10 years

In [13]:
########## ########## ##########

# works after running a second time. why?
# should i write a script to try twice?

########## ########## ##########

all_movies = []
for i in years:
    url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected={}&sort=desc'.format(i)

    # bypass errors using Request/urlopen changing user-agent
    headers = {'User-Agent': 'Firefox'}
    req = Request(url, headers=headers)

    page = urlopen(req).read()

    # beautifulsoup object "soup" of webpage
    soup = BS(page, 'html.parser')
    movies = soup.find_all('td', class_='clamp-summary-wrap')

    # append list of tuples with results of each year's iteration
    all_movies += parse_results(movies)
    time.sleep(12) # don't over-ping metacritic site


In [ ]:
movie_tuples

In [8]:
all_movies

[('Parasite', datetime.datetime(2019, 10, 11, 0, 0), '2019', '95'),
 ('Marriage Story', datetime.datetime(2019, 11, 6, 0, 0), '2019', '94'),
 ('The Irishman', datetime.datetime(2019, 11, 1, 0, 0), '2019', '94'),
 ('Portrait of a Lady on Fire',
  datetime.datetime(2019, 12, 6, 0, 0),
  '2019',
  '93'),
 ('Apocalypse Now: Final Cut',
  datetime.datetime(2019, 8, 15, 0, 0),
  '2019',
  '92'),
 ('The Souvenir', datetime.datetime(2019, 5, 17, 0, 0), '2019', '92'),
 ('Uncut Gems', datetime.datetime(2019, 12, 13, 0, 0), '2019', '91'),
 ('For Sama', datetime.datetime(2019, 7, 26, 0, 0), '2019', '90'),
 ('The Farewell', datetime.datetime(2019, 7, 12, 0, 0), '2019', '89'),
 ("Long Day's Journey Into Night",
  datetime.datetime(2019, 4, 12, 0, 0),
  '2019',
  '89'),
 ('Pain and Glory', datetime.datetime(2019, 10, 4, 0, 0), '2019', '88'),
 ('Apollo 11', datetime.datetime(2019, 3, 1, 0, 0), '2019', '88'),
 ('Genèse (Genesis)', datetime.datetime(2019, 8, 23, 0, 0), '2019', '87'),
 ('In Fabric', date

### Clean/parse data in order to make it insertable into DB

In [4]:
# loop through list of 100 movies and pull relevant data

def parse_results(movies):
    
    movie_tuples = []

    for movie in movies:
        
        movie_title = movie.find('h3').text

        movie_date = movie.find('div', class_='clamp-details').span.text
        format_date = datetime.strptime(movie_date, '%B %d, %Y') # convert to datetime

        movie_year = movie.find('div', class_='clamp-details').span.text[-4:]
        
        # "perfect metascore" has a different class name than any other positive score
        try:
            movie_metascore = movie.find('div', class_='metascore_w large movie positive').text
        except:
            movie_metascore = 100

        # add details to a tuple to add to MYSQL db
        movie_tuple = (movie_title, format_date, movie_year, movie_metascore)
        movie_tuples.append(movie_tuple)
    
    return movie_tuples

# movie_tuples
# movie_date
# datetime_obj
# format_date

In [14]:
########## ########## ##########

# Attempt at reworking date format for insertion into DB

########## ########## ##########

# datetime_obj = datetime.strptime(movie_date, '%B %d, %Y')
# foo = movie_tuples[0][1]
# x = datetime.datetime.strptime(foo, '%B %d, %Y')
# x
# datetime_obj
# movie_dates[0]

In [15]:
print(movie_tuples[0][1])

NameError: name 'movie_tuples' is not defined

In [9]:
# Import packages to check data insertion into DB and DB login info file

import configProj1
import mysql.connector
from mysql.connector import errorcode

In [10]:
## connecting to the database using 'connect()' method

cnx = mysql.connector.connect(
    host = configProj1.host,
    user = configProj1.user,
    passwd = configProj1.password,
    database = 'Movies'
)

cursor = cnx.cursor()

In [ ]:
# create table in MYSQL

# TABLES = {}
# TABLES['Metascore'] = """
#                 CREATE TABLE Metascore 
#                 (
#                     title varchar(50) NOT NULL UNIQUE,
#                     release_date date NOT NULL,
#                     metascore int(10) NOT NULL,
#                     PRIMARY KEY (title)
#                 );
#                 """

In [ ]:
# check if table is created using errorcode library

# for table_name in TABLES:
#     table_description = TABLES[table_name]
#     try:
#         print("Creating table {}: ".format(table_name), end='')
#         cursor.execute(table_description)
#     except mysql.connector.Error as err:
#         if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
#             print("already exists.")
#         else:
#             print(err.msg)
#     else:
#         print("OK")

In [ ]:
# add column "release year"
# cursor.execute("""
#                 ALTER TABLE Metascore
#                 ADD COLUMN release_year int(7) NOT NULL;
#                 """)

# change table name to METACRITIC
# cursor.execute("""
#                 ALTER TABLE Metascore
#                 RENAME TO Metacritic;
#                 """)
# cursor.execute("""
#                 DROP TABLE Metascore;
#                 """)


In [11]:
# loop over all elements in output list of 1000 tuples and insert into db
for i in range(0,len(all_movies)-1):
    try:
        stmt = "INSERT INTO Metacritic (title, release_date, release_year, metascore) VALUES (%s,%s,%s,%s)"
        cursor.execute(stmt, all_movies[i])
    
    # 5/1000 movies did not make the db because of the title being a duplicate
    except mysql.connector.Error as err:
        print(err)
        print(all_movies[i])

# DON'T FORGET TO COMMIT
cnx.commit()

1062 (23000): Duplicate entry 'Western' for key 'title'
('Western', datetime.datetime(2015, 9, 25, 0, 0), '2015', '89')
1062 (23000): Duplicate entry 'Leviathan' for key 'title'
('Leviathan', datetime.datetime(2013, 3, 1, 0, 0), '2013', '81')
1062 (23000): Duplicate entry 'Elena' for key 'title'
('Elena', datetime.datetime(2012, 5, 18, 0, 0), '2012', '87')
1062 (23000): Duplicate entry 'Loveless' for key 'title'
('Loveless', datetime.datetime(2011, 2, 18, 0, 0), '2011', '75')
1062 (23000): Duplicate entry 'The Square' for key 'title'
('The Square', datetime.datetime(2010, 4, 9, 0, 0), '2010', '75')


In [16]:
# close cursor, close connection
cursor.close()
cnx.close()